# **EDA for up_event variable**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
# feature_engineering
import re

# ploting
import matplotlib.pyplot as plt
# model initial:
import statsmodels.api as sm

#testing:
import unittest

In [2]:
#Set import folder
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
# Load train logs
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
# Set the id and event id as index:
train_logs_indexed = train_logs.set_index(['id', 'event_id'])
# Load train scores
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(6)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1


In [5]:
train_logs.tail(3)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240
8405897,fff05981,3619,2070065,2070133,68,Input,.,.,.,1029,240


In [6]:
train_logs.shape

(8405898, 11)

In [7]:
train_logs.up_event.describe()

count     8405898
unique        130
top             q
freq      5364789
Name: up_event, dtype: object

In [8]:
score_60 = scores[scores == 6.0].index.values
score_05 = scores[scores == 0.5].index.values


In [9]:
train_logs.up_event.unique()[:25]

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'm', 'Rightclick', 'i', 'o', 't', '=', 'a'],
      dtype=object)

In [10]:
train_logs.up_event.unique().shape

(130,)

In [29]:
t = scores[scores == .5].index.values
for i in range(t.shape[0]):
    v = train_logs_indexed.loc[t[i]]['up_event'].equals(train_logs_indexed.loc[t[i]]['down_event'])
    print(v)

True
True
True
True
True


In [30]:
train_logs['up_event'].unique()

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'm', 'Rightclick', 'i', 'o', 't', '=', 'a',
       'CapsLock', 'Control', 'c', 'v', '/', 'Delete', ':', 'z', '[', '$',
       '(', ')', '+', 'Home', 'End', '\\', 'Meta', '*', '&',
       'AudioVolumeMute', 'x', '!', 'Insert', 'MediaPlayPause', 'w',
       'NumLock', '%', 'V', 'b', '>', 'Alt', 'AudioVolumeUp',
       'ContextMenu', 'AudioVolumeDown', 'n', 'e', '<', 'PageDown', ']',
       'Middleclick', '@', 'F12', 'u', 'j', 's', '\x96', 'Dead', 'y', '{',
       'ScrollLock', '¿', 'p', 'Process', '}', 'MediaTrackPrevious',
       'MediaTrackNext', 'F3', '^', 'Unidentified', 'Cancel', 'h', '2',
       'd', 'r', '`', '\x9b', 'f', 'g', '#', '~', 'PageUp', 'l', 'T', 'A',
       'S', 'ModeChange', '_', 'Escape', 'F11', 'Unknownclick',
       'AltGraph', 'F10', 'F15', 'Clear', 'OS', 'C', 'Ä±', 'M', '|',
       'â\x80\x93', 

In [33]:
t = scores[scores == .5].index.values
for i in range(t.shape[0]):
    x = train_logs_indexed.loc[t[i]]['up_event'].unique()
    print(x)

['Leftclick' 'q' 'Backspace' 'Space' '.' 'CapsLock' 'Enter' ',']
['Leftclick' 'q' 'Backspace' 'CapsLock' 'Space' '.' 'Enter' '\\']
['Leftclick' 'Shift' 'q' 'Space' 'Backspace' '.' 'Enter' 'ArrowUp' 'Tab']
['Leftclick' 'Space' 'Shift' 'q' 'Backspace' 'Enter' ':' ']' '.' "'" ','
 'Rightclick']
['Leftclick' 'q' 'Space' 'Backspace' '.' 'Shift' 'F2' 'Delete' 'Meta' ',']


In [81]:
# eda up_event
# Based on the above, check for "MediaPlayPause"
def find_clicked(x, st:str):
    has_string = 0
    for event in x:
        if(event == st):
            has_string = 1
            break
    return has_string

ue = train_logs.groupby(['id'])['up_event'].aggregate([('ue_1',lambda x: find_clicked(x,"Backspace")),
                                                      ('ue_2', lambda x: find_clicked(x,"Shift")),
                                                      ('ue_3', lambda x: find_clicked(x,"Tab")),
                                                      ('ue_4', lambda x: find_clicked(x,"CapsLock")),
                                                      ('ue_5', lambda x: find_clicked(x,"\'")),
                                                      ('ue_6', lambda x: find_clicked(x,":")),
                                                      ])
ue.head(3)

,ue_1,ue_2,ue_3,ue_4,ue_5,ue_6
id,,,,,,
001519c8,1,1,0,0,1,0
0022f953,1,1,1,0,1,0
0042269b,1,1,0,0,0,0


In [82]:
z = pd.merge(ue, scores, left_index = True, right_index = True)
z.corr()['score']

ue_1     0.018195
ue_2     0.284764
ue_3     0.147044
ue_4    -0.213481
ue_5     0.326035
ue_6     0.121229
score    1.000000
Name: score, dtype: float64